In [1]:
import pandas as pd
import numpy as np
import nafot
import os
from tqdm import tqdm_notebook
import multiprocessing as mp
import time
import datetime

### Get the sample files list

In [2]:
file = pd.read_csv('../data/samples/new samples/with stat area/sample_00with_stat.csv')

In [18]:
file.drop('Unnamed: 0', axis=1, inplace=True)

In [45]:
# Remove records without stat area
file.dropna(inplace=True)

In [54]:
os.listdir('../data/samples/new samples/with stat area')

['.ipynb_checkpoints',
 'fix files index.ipynb',
 'sample_00with_stat.csv',
 'sample_01with_stat.csv',
 'sample_02with_stat.csv',
 'sample_03with_stat.csv',
 'sample_04with_stat.csv',
 'sample_05with_stat.csv',
 'sample_06with_stat.csv',
 'sample_07with_stat.csv',
 'sample_08with_stat.csv',
 'sample_09with_stat.csv',
 'sample_10with_stat.csv',
 'sample_11with_stat.csv',
 'sample_12with_stat.csv',
 'sample_13with_stat.csv']

In [ ]:
# Get the file names
sample_files_names = os.listdir('../data/samples/new samples/with stat area')

for file_name in tqdm_notebook(sample_files_names, 'files'):
    

In [ ]:
%%time
# Create a pool of 24 processes
pool = mp.Pool(24)

# Get the file names
# sample_files_names = os.listdir('../data/samples/new samples/with stat area')

# for file_name in tqdm_notebook(sample_files_names[0:2], 'files'):
    
# Get the list of the unique imsi
imsi_list = file.imsi.unique()

# Devide the imsi list into 24 equal-sized chunks 
imsi_chunks = np.split(imsi_list, np.arange(0,len(imsi_list), int(len(imsi_list)/24)+1))[1:]

# Create data frames of the data by imsi in the chunks
imsi_data = [file[file.imsi.isin(chunk)] for chunk in imsi_chunks]

# start time
start_time = time.time() 

# creating func for multiprocess
func_list = []
for data in imsi_data:
#     f = pool.apply_async(nafot.get_home_stat_area, [data, False])
    f = pool.apply_async(get_home_data_and_visit_matrix, [data])
    func_list.append(f)

# end time
end_time = time.time()
# run time
run_time = (end_time - start_time)
# print(f'{file_name}: functions created. run time:{str(datetime.timedelta(seconds=run_time))}')

# Process chunks in parallel
# start time
start_time = time.time() 
# print(f'{file_name} start time: {time.ctime()}')

dfs_home =  []
final_visit_mat_raw = np.zeros((data.shape))
for f in func_list:
    # processing
    home_data, visit_matrix_raw = f.get()
    # update
    dfs_home.append(home_data)
    final_visit_mat_raw += visit_matrix_raw

# end time
end_time = time.time()
# Getting thr run time
run_time = (end_time - start_time)   
# print(f'{file_name}: Process completed. Processing time: {str(datetime.timedelta(seconds=run_time))}')

# Concat chunks
final_df = pd.concat(dfs)
# print(f'{file_name}: chunks concatenated. final_df shape: {final_df.shape}')
# final_df.to_csv(f'../data/samples/new samples/home area/{file_name[:-4]}home_area.csv', index=False)

In [53]:
%%time
a = nafot.get_home_stat_area(file, path=False)

Wall time: 12min 33s


In [56]:
a[a.home_stat_area != 'NotDetermined'].shape

(8056, 2)

In [60]:
aa = a[a.imsi.isin(imsi_data[0].imsi)]

In [61]:
%%time
b = nafot.calculate_visit_matrix(imsi_data[0], aa)

Wall time: 54 s


In [64]:
%%time
c,d = get_home_data_and_visit_matrix(imsi_data[0])

Wall time: 1min 5s


In [62]:
def get_home_data_and_visit_matrix(loc_data):
    home_data = nafot.get_home_stat_area(loc_data, False)
    visit_matrix_raw = nafot.calculate_visit_matrix(loc_data, home_data)
    
    return home_data, visit_matrix_raw

In [ ]:
sample_files_names = os.listdir('../data/samples/new samples/')

# Create a pool of 24 processes
pool = mp.Pool(24)

for file_name in tqdm_notebook(sample_files_names[0:2], 'files'):
    
    # read the file in chunks
    reader = pd.read_csv(f'../data/samples/new samples/{file_name}', chunksize=6500000)
    
    # start time
    start_time = time.time() 
    
    # creating func for multiprocess
    func_list = []
    for data in reader:
        f = pool.apply_async(nafot.add_stat_area_column, [data, False])
        func_list.append(f)
    
    # end time
    end_time = time.time()
    # run time
    run_time = (end_time - start_time)
    print(f'{file_name}: functions created. run time:{str(datetime.timedelta(seconds=run_time))}')
    
    # Process chunks in parallel
    # start time
    start_time = time.time() 
    print(f'{file_name} start time: {time.ctime()}')
   
    dfs = []
    for f in func_list:
        # processing
        dfs.append(f.get())
    
    # end time
    end_time = time.time()
    # Getting thr run time
    run_time = (end_time - start_time)   
    print(f'{file_name}: Process completed. Processing time: {str(datetime.timedelta(seconds=run_time))}')

    # Concat chunks
    final_df = pd.concat(dfs)
    print(f'{file_name}: chunks concatenated. final_df shape: {final_df.shape}')
    final_df.to_csv(f'../data/samples/new samples/with stat area/{file_name[:-4]}with_stat.csv', index=False)

A Jupyter Widget

sample_12.csv: functions created. run time:0:03:53.043694
sample_12.csv start time: Sun Jan  7 09:38:14 2018
sample_12.csv: Process completed. Processing time: 15:55:45.863239
sample_12.csv: chunks concatenated. final_df shape: (149986027, 7)
sample_13.csv: functions created. run time:0:03:48.274868
sample_13.csv start time: Mon Jan  8 01:56:16 2018
